# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read-in the .csv from Part 1
weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

#Display data
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed
0,Tiznit,44,MA,1580145602,41,29.58,-9.50,59.79,4.23
1,Puerto Escondido,5,MX,1580145603,57,15.85,-97.07,80.60,6.26
2,Ozark,90,US,1580145604,66,37.02,-93.21,52.00,4.12
3,Ancud,75,CL,1580145604,48,-41.87,-73.82,64.40,12.75
4,Tasiilaq,90,GL,1580145605,54,65.61,-37.64,30.20,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Import Google API key
gmaps.configure(api_key=g_key)

In [4]:
#Define variables and parameters for headmap
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure(center=(20.0,0.0),zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity=50, point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Drop NaN results
weather_df = weather_df.dropna()

#Define weather criteria
weather_criteria = weather_df[(weather_df['Maximum Temperature'] > 55) & 
                              (weather_df['Maximum Temperature']< 70) & 
                              (weather_df['Humidity'] < 40) &
                              (weather_df['Wind Speed'] < 10)&
                              (weather_df['Cloudiness'] < 50)]

weather_criteria.reset_index()

,index,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed
0,134,Adre,3,TD,1580145696,27,13.47,22.20,61.29,4.32
1,308,Marzuq,0,YE,1580145823,32,14.40,46.47,58.82,2.53
2,354,Umm Lajj,0,SA,1580145863,32,25.02,37.27,61.12,4.23
3,370,Bahir Dar,0,ET,1580145875,23,11.59,37.39,65.34,7.85


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create new data frame with specific columns from criteria results
hotel_df = pd.DataFrame(weather_criteria,columns=['City','Country','Latitude','Longitude'])

#Add a new column for "Hotel Name"
hotel_df['Hotel Name']= ""

#Drop NaN results
hotel_df.dropna()

hotel_df.reset_index()

,index,City,Country,Latitude,Longitude,Hotel Name
0,134,Adre,TD,13.47,22.20,
1,308,Marzuq,YE,14.40,46.47,
2,354,Umm Lajj,SA,25.02,37.27,
3,370,Bahir Dar,ET,11.59,37.39,


In [7]:
#Create empty list for Hotel Name column and parameters to search
hotel_name=[]
target_type = "hotel"
radius = 5000

#Define parameters
params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}

#For loop to search for hotels in locations meeting weather criteria
for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
      
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json=response.json()

    try: 
        hotel_name.append(response_json['results'][0]['name'])
    
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    
        print(f"The nearest hotel to {row[0]} is {response_json['results'][0]['name']}")  
    
    except (KeyError, IndexError):
        print("Cannot locate a hotel - skipping.")

The nearest hotel to Adre is Adre
The nearest hotel to Marzuq is Al Hujayrah
The nearest hotel to Umm Lajj is Umluj
The nearest hotel to Bahir Dar is Bahir Dar


In [8]:
#Drop any potential NaN results
hotel_df = hotel_df.dropna()
hotel_df.reset_index()

,index,City,Country,Latitude,Longitude,Hotel Name
0,134,Adre,TD,13.47,22.20,Adre
1,308,Marzuq,YE,14.40,46.47,Al Hujayrah
2,354,Umm Lajj,SA,25.02,37.27,Umluj
3,370,Bahir Dar,ET,11.59,37.39,Bahir Dar


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = weather_criteria[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(20.0,0.0),zoom_level=1)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))